In [1]:
Output = ('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1011. NC Well Metals/P1011.2. Analyses/P1011.2.2. Metal Prediction/Output')
cur_date = "060524"

library(readxl)
library(writexl)
library(openxlsx)
library(lubridate)
library(tidyverse)
library(gtsummary)
library(caret)
library(e1071)
library(Hmisc)
library(randomForest)
library(pROC)
library(themis)

# reading in file
well_data = data.frame(read_excel("Input/Imputed_Well_Data_020924.xlsx")) 


Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr   1.1.3     ✔ readr   2.1.4
✔ forcats 1.0.0     ✔ stringr 1.5.0
✔ ggplot2 3.4.3     ✔ tibble  3.2.1
✔ purrr   1.0.2     ✔ tidyr   1.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loading required package: lattice


Attaching package: ‘caret’


The following object is masked from ‘package:purrr’:

    lift



Attaching package: ‘Hmisc’


The following object is masked from ‘package:e1071’:

    impute


The following objects are masked from ‘package:dplyr’:

    src, summarize


The following objects are masked from ‘package:base’:

    format.pval, units


rand

In [2]:
head(well_data)

,Tax_ID,Health_Dept_ID,Permit_No,Water_Sample_Date,Casing_Depth,Well_Depth,Static_Water_Depth,Flow_Rate,pH,Metal,Longitude,Latitude,Geologic_Terrane,Rock_Type,Soil_Type_Condensed,Landuse_Condensed,Elevation,Stream_Distance,Concentration,Detect_Concentration
,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>
1,4006015,344,09-192,NA,97,450,35,20.0,7.4,Ar,-80.55427,34.87224,CSB,MV,H,C,675.174,654.6816,2.007062,ND
2,04009002C,1525,14-18,12/7/10,65,300,32,2.0,7.0,Ar,-80.55676,34.87109,CSB,MV,D,C,678.113,454.1912,1.912321,ND
3,04030015C,1525,14-18,8/27/12,65,300,32,2.0,8.1,Ar,-80.55676,34.87109,CSB,MV,D,C,469.708,454.1912,7.000000,D
4,04030015J,1525,14-18,4/5/10,65,300,32,2.0,8.1,Ar,-80.55676,34.87109,CSB,MV,H,F,470.293,454.1912,1.153921,ND
5,04030020H,234,09-147,10/25/10,52,125,36,20.0,7.6,Ar,-80.55220,34.86012,CSB,MV,H,F,470.293,918.3419,1.675360,ND
6,4030041,1515,14-04,3/2/16,47,275,34,2.5,8.2,Ar,-80.56423,34.88559,CSB,MV,H,D,470.293,512.3955,14.000000,D


## Metal Correlation

In [3]:
# assessing collinearity between the metals
as_df = well_data %>% 
    filter(Metal == "Ar")
mn_df = well_data %>% 
    filter(Metal == "Mn")
cr_df = well_data %>% 
    filter(Metal == "Cr")

cor.test(as_df$Concentration, mn_df$Concentration)
cor.test(as_df$Concentration, cr_df$Concentration)
cor.test(mn_df$Concentration, cr_df$Concentration)


	Pearson's product-moment correlation

data:  as_df$Concentration and mn_df$Concentration
t = -0.058171, df = 713, p-value = 0.9536
alternative hypothesis: true correlation is not equal to 0
95 percent confidence interval:
 -0.07548740  0.07115379
sample estimates:
         cor 
-0.002178519 



	Pearson's product-moment correlation

data:  as_df$Concentration and cr_df$Concentration
t = -0.48146, df = 713, p-value = 0.6303
alternative hypothesis: true correlation is not equal to 0
95 percent confidence interval:
 -0.09122823  0.05536624
sample estimates:
        cor 
-0.01802788 



	Pearson's product-moment correlation

data:  mn_df$Concentration and cr_df$Concentration
t = -1.3792, df = 713, p-value = 0.1683
alternative hypothesis: true correlation is not equal to 0
95 percent confidence interval:
 -0.12443367  0.02182011
sample estimates:
        cor 
-0.05158335 


None of the metals are highly correlated. 

# Prediction of Contaminated Wells using Supervised ML

Using `Water_Sample_Date`, `Casing_Depth`, `Well_Depth`, `Static_Water_Depth`, `pH`, `Flow_Rate`, `Stream_Distance`, `Elevation`, `Geologic_Terrane` or `Rock_Type`, `Soil_Type_Condensed`, `Landuse_Condensed`, `Latitude`, and `Longitude` to predict whether Mn concentration falls above or below the EPA's lifetime Health Advisory Limit (HAL) (< or >= 300 ppb). RF and SVM models will be built to predict concentration this binarized concentration. 

Additionally, models will be run using a different combination of the aforementioned predictors detailed into the following use cases below:

1. All Data (using all 13 features)
2. All variables excluding latitude and longitude (Using this to see if the prediction's accuracy is retained removing those coordinates to make the results more generalizable to areas outside of Union County, NC.)
3. Well Chacteristics only (casing depth, pH, flow rate, well depth, and static water depth)
4. Health Department lens (rock type, latitude, and longitude)

Starting by creating an additional variable for above and below 300 ppb and formatting the df for input into ML models.

In [4]:
manganese_data = well_data %>%
    mutate(MCL = relevel(factor(ifelse(Concentration >= 300, 1, 0)), ref = "0"),
          # converting water sample date from a character to a date type 
          Water_Sample_Date = mdy(Water_Sample_Date)) %>%
    # filtering for manganese only and removing some rows with missing data
    filter(Metal == "Mn" & Landuse_Condensed != "NA") %>%
    select(-c("Metal", "Detect_Concentration"))

# changing data types
manganese_data$Geologic_Terrane = factor(manganese_data$Geologic_Terrane)
manganese_data$Rock_Type = factor(manganese_data$Rock_Type)
manganese_data$Soil_Type_Condensed = factor(manganese_data$Soil_Type_Condensed)
manganese_data$Landuse_Condensed = factor(manganese_data$Landuse_Condensed)

head(manganese_data)

Warning message:
“There was 1 warning in `mutate()`.
ℹ In argument: `Water_Sample_Date = mdy(Water_Sample_Date)`.
Caused by warning:
!  6 failed to parse.”


,Tax_ID,Health_Dept_ID,Permit_No,Water_Sample_Date,Casing_Depth,Well_Depth,Static_Water_Depth,Flow_Rate,pH,Longitude,Latitude,Geologic_Terrane,Rock_Type,Soil_Type_Condensed,Landuse_Condensed,Elevation,Stream_Distance,Concentration,MCL
,<chr>,<dbl>,<chr>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<fct>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<fct>
1,04009002C,344,09-192,2010-12-07,97,450,35,20.0,7.4,-80.55427,34.87224,CSB,MV,H,C,678.113,654.6816,21.47573,0
2,04030015C,1525,14-18,2012-08-27,65,300,32,2.0,7.0,-80.55676,34.87109,CSB,MV,D,C,469.708,454.1912,23.35497,0
3,04030015J,1525,14-18,2010-04-05,65,300,32,2.0,8.1,-80.55676,34.87109,CSB,MV,D,C,470.293,454.1912,1300.00000,1
4,04030020H,1525,14-18,2010-10-25,65,300,32,2.0,8.1,-80.55676,34.87109,CSB,MV,H,F,470.293,454.1912,850.00000,1
5,4030041,234,09-147,2016-03-02,52,125,36,20.0,7.6,-80.55220,34.86012,CSB,MV,H,F,470.293,918.3419,22.76362,0
6,4033001,1515,14-04,2011-12-06,47,275,34,2.5,8.2,-80.56423,34.88559,CSB,MV,H,D,451.806,512.3955,90.00000,0


In [5]:
# original number of records
dim(well_data)

# records kept for analysis
dim(manganese_data)

[1] 2145   20

[1] 714  19

# Summary Statistics
Determining if there are any signficiant differenes between the predictor variables for each outcome variable.

In [6]:
manganese_data %>%
  tbl_summary(by = MCL, missing = "no", 
  include = colnames(manganese_data[c(4:17)]), 
              statistic = list(all_continuous() ~ "{mean} ({sd})",
                               all_categorical() ~ "{n} ({p}%)")) %>%
  add_n() %>% 
  #add_overall() %>%
  add_p(test = list(all_continuous() ~ "t.test",
                    all_categorical() ~ "chisq.test")) %>% # adding p value from anova
  as_tibble() %>%
  write_xlsx(., "Output/Table1b.xlsx")

There was an error in 'add_p()/add_difference()' for variable 'Water_Sample_Date', p-value omitted:
Error in Math.Date(mx): abs not defined for "Date" objects

Warning for variable 'Rock_Type':
simpleWarning in stats::chisq.test(x = structure(c(4L, 4L, 4L, 4L, 4L, 4L, 4L, : Chi-squared approximation may be incorrect

Warning for variable 'Soil_Type_Condensed':
simpleWarning in stats::chisq.test(x = structure(c(2L, 1L, 1L, 2L, 2L, 2L, 3L, : Chi-squared approximation may be incorrect

Warning for variable 'Landuse_Condensed':
simpleWarning in stats::chisq.test(x = structure(c(1L, 1L, 1L, 3L, 3L, 2L, 1L, : Chi-squared approximation may be incorrect



Although the p values are significant between some of the variables, there is a high level of class imbalance which is likely to affect model performance. Therefore, class imbalance will be addressed using SMOTE. 

In [7]:
# creating dfs for each use case
# dropped 2 rows that had missing dates
manganese_case_1_df = drop_na(manganese_data[,c(4:11,13:17,19)])
manganese_case_2_df = drop_na(manganese_data[,c(4:9,13:17,19)])
manganese_case_3_df = drop_na(manganese_data[,c(5:9,19)])
manganese_case_4_df = drop_na(manganese_data[,c(10,11,13,19)])

head(manganese_case_1_df)

,Water_Sample_Date,Casing_Depth,Well_Depth,Static_Water_Depth,Flow_Rate,pH,Longitude,Latitude,Rock_Type,Soil_Type_Condensed,Landuse_Condensed,Elevation,Stream_Distance,MCL
,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<fct>,<fct>,<dbl>,<dbl>,<fct>
1,2010-12-07,97,450,35,20.0,7.4,-80.55427,34.87224,MV,H,C,678.113,654.6816,0
2,2012-08-27,65,300,32,2.0,7.0,-80.55676,34.87109,MV,D,C,469.708,454.1912,0
3,2010-04-05,65,300,32,2.0,8.1,-80.55676,34.87109,MV,D,C,470.293,454.1912,1
4,2010-10-25,65,300,32,2.0,8.1,-80.55676,34.87109,MV,H,F,470.293,454.1912,1
5,2016-03-02,52,125,36,20.0,7.6,-80.55220,34.86012,MV,H,F,470.293,918.3419,0
6,2011-12-06,47,275,34,2.5,8.2,-80.56423,34.88559,MV,H,D,451.806,512.3955,0


# Random Forest
- an ensemble learning method operating by constructing a multitude of decision trees at training time, which uses multiple methods to obtain a better predictive performance and includes bagging and random forest
- algorithm uses a bootstrop aggregation, to reduce overfitting the training datset but only a subset of the features are used hence decorrelation of predictors

In [8]:
rf_classification = function(dataset, outcome, pred_outcome, use_case){
    # setting for reproducibility
    set.seed(12)
    # splitting data into training and testing sets
    dataset_index = createFolds(dataset[[outcome]], k = 5) # 5 fold CV
    
    ntree_values = c(50, 250, 500) # number of trees 
    p = dim(dataset)[2] - 1 # number of variables in dataset
    mtry_values = c(sqrt(p), p/2, p/3) # number of predictors

    metrics = data.frame()
    variable_importance_df = data.frame()
    roc_objects = c()
    threshold_data = data.frame()
    
    for (i in 1:length(dataset_index)){
        
        data_train = dataset[-dataset_index[[i]],]
        # using SMOTE to balance the class distribution
        balanced_data_train = smotenc(data_train, outcome)
        data_test = dataset[dataset_index[[i]],]

        # will use ntree and mtry values to determine which combination yields the smallest MSE
        reg_rf_pred_tune = list()
        rf_OOB_errors = list()
        rf_error_df = data.frame()
        for (j in 1:length(ntree_values)){
            for (k in 1:length(mtry_values)){
                reg_rf_pred_tune[[k]] = randomForest(as.formula(paste0(outcome, "~.")), data = balanced_data_train, 
                                                     ntree = ntree_values[j], mtry = mtry_values[k])
                rf_OOB_errors[[k]] = data.frame("Tree Number" = ntree_values[j], "Variable Number" = mtry_values[k], 
                                       "OOB_errors" = reg_rf_pred_tune[[k]]$err.rate[ntree_values[j],1])
                rf_error_df = rbind(rf_error_df, rf_OOB_errors[[k]])
            }
        }

        # finding the lowest OOB error using best number of predictors at split and refitting OG tree
        best_oob_errors = which(rf_error_df$OOB_errors == min(rf_error_df$OOB_errors))

        reg_rf = randomForest(as.formula(paste0(outcome, "~.")), data = balanced_data_train,
                               ntree = rf_error_df$Tree.Number[min(best_oob_errors)],
                               mtry = rf_error_df$Variable.Number[min(best_oob_errors)])

        # predicting on test set
        data_test[[pred_outcome]] = predict(reg_rf, newdata = data_test, type = "response")
        
        matrix = confusionMatrix(data = data_test[[pred_outcome]], reference = data_test[[outcome]], 
                                     positive = "1")

        # calculating AUC
        auc = auc(response = data_test[[outcome]], predictor = factor(data_test[[pred_outcome]], ordered = TRUE))
        
        # calculating values to plot ROC curve later
        roc_obj = roc(response = data_test[[outcome]], predictor = factor(data_test[[pred_outcome]], ordered = TRUE))

        # Return max Youden's index, with specificity and sensitivity
        best_thres_data = data.frame(coords(roc_obj, x = "best", best.method = c("youden", "closest.topleft")))
        threshold_data = rbind(threshold_data, best_thres_data)
        
        #extracting accuracy, sens, spec, PPV, NPV to take mean later
        matrix_values = data.frame(t(c(matrix$byClass[11])), t(c(matrix$byClass[1:4])), auc)
        
        # extracting variable importance
        var_importance_values = data.frame(importance(reg_rf)) %>%
            rownames_to_column(var = "Predictor")
        variable_importance_df = rbind(variable_importance_df, var_importance_values)
   
        # adding values to df
        metrics = rbind(metrics, matrix_values)
        
    }
    
    # taking averages/sd 
    metrics = metrics %>%
        summarise(`Balanced Accuracy` = mean(Balanced.Accuracy), Sensitivity = mean(Sensitivity), 
              Specificity = mean(Specificity), PPV = mean(Pos.Pred.Value), NPV = mean(Neg.Pred.Value), 
              AUC = mean(auc))
    
    variable_importance_df = variable_importance_df %>%
        group_by(Predictor) %>%
        summarise(MeanDecreaseGini = mean(MeanDecreaseGini)) %>%
        # sorting by most important variables
        arrange(-MeanDecreaseGini) %>%
        mutate(`Use_Case` = use_case)
  
    # return training set, matrix, variable importance values, (last) roc object, best threshold data
    return(list(balanced_data_train, metrics, variable_importance_df, roc_obj, threshold_data))

}

In [9]:
# calling fn
rf_values_manganese_case_1 = rf_classification(manganese_case_1_df, "MCL", "pred_MCL",1)
rf_values_manganese_case_2 = rf_classification(manganese_case_2_df, "MCL", "pred_MCL",2)
rf_values_manganese_case_3 = rf_classification(manganese_case_3_df, "MCL", "pred_MCL",3)
rf_values_manganese_case_4 = rf_classification(manganese_case_4_df, "MCL", "pred_MCL",4)

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting leve

In [10]:
# viewing results
rf_confusion_matrix = data.frame(Model = "RF Classification", 
                                 Use_Case = 1:4, Kernel = NA, 
                                 rbind(rf_values_manganese_case_1[[2]], rf_values_manganese_case_2[[2]],
                                 rf_values_manganese_case_3[[2]],rf_values_manganese_case_4[[2]])) 

rf_confusion_matrix

# viewing most significant features
rf_values_manganese_case_1[[3]]

Model,Use_Case,Kernel,Balanced.Accuracy,Sensitivity,Specificity,PPV,NPV,AUC
<chr>,<int>,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
RF Classification,1,NA,0.6974874,0.4895425,0.9054323,0.4279656,0.9269879,0.6974874
RF Classification,2,NA,0.6386978,0.3751634,0.9022323,0.3418386,0.9116613,0.6386978
RF Classification,3,NA,0.6311122,0.3516340,0.9105905,0.3668854,0.9091508,0.6311122
RF Classification,4,NA,0.6993815,0.5313725,0.8673905,0.3579872,0.9302196,0.6993815


Predictor,MeanDecreaseGini,Use_Case
<chr>,<dbl>,<dbl>
Longitude,127.10061,1
Casing_Depth,65.64029,1
Latitude,54.52511,1
Flow_Rate,35.66445,1
pH,35.43432,1
Well_Depth,33.21099,1
Elevation,29.94335,1
Static_Water_Depth,27.33723,1
Water_Sample_Date,26.62692,1


# Support Vector Machine (SVM)
- supervised learning models that can predict continuous (regression) or grouped/dichotomous (classification) outcomes
- predicts by projecting them onto a high dimensional space and uses kernels to make the data more separable (unfortunately makes interpretability of model results more difficult)
- does a better job at handling a large number of predictors since p > n
- Compared to other classification algorithms, this approach can reliably classify chemicals while avoiding overfitting and reducing susceptibility to noisy or meaningless data

In [11]:
# the caret fn prefers for the outcome variable to have letters instead of numbers so reformatting the df
manganese_data = manganese_data %>%
    mutate(MCL = relevel(factor(ifelse(Concentration >= 300, "D", "ND")), ref = "ND"))

In [12]:
# creating dfs for each use case
# dropped 2 rows that had missing dates
manganese_case_1_df = drop_na(manganese_data[,c(4:11,13:17,19)])
manganese_case_2_df = drop_na(manganese_data[,c(4:9,13:17,19)])
manganese_case_3_df = drop_na(manganese_data[,c(5:9,19)])
manganese_case_4_df = drop_na(manganese_data[,c(10,11,13,19)])

head(manganese_case_1_df)

,Water_Sample_Date,Casing_Depth,Well_Depth,Static_Water_Depth,Flow_Rate,pH,Longitude,Latitude,Rock_Type,Soil_Type_Condensed,Landuse_Condensed,Elevation,Stream_Distance,MCL
,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<fct>,<fct>,<dbl>,<dbl>,<fct>
1,2010-12-07,97,450,35,20.0,7.4,-80.55427,34.87224,MV,H,C,678.113,654.6816,ND
2,2012-08-27,65,300,32,2.0,7.0,-80.55676,34.87109,MV,D,C,469.708,454.1912,ND
3,2010-04-05,65,300,32,2.0,8.1,-80.55676,34.87109,MV,D,C,470.293,454.1912,D
4,2010-10-25,65,300,32,2.0,8.1,-80.55676,34.87109,MV,H,F,470.293,454.1912,D
5,2016-03-02,52,125,36,20.0,7.6,-80.55220,34.86012,MV,H,F,470.293,918.3419,ND
6,2011-12-06,47,275,34,2.5,8.2,-80.56423,34.88559,MV,H,D,451.806,512.3955,ND


In [13]:
tunegrid_linear_svm = data.frame(C = seq(from = 0.01, to = 0.1, by = 0.01))

svm_classification = function(dataset, model, outcome, pred_outcome, tunegrid, use_case){

    #setting seed for reproducibility
    set.seed(12)
    
    # 5 fold CV first
    control = trainControl(method = 'cv', number = 5, search = 'grid', classProbs = TRUE, savePredictions = TRUE,
        summaryFunction = twoClassSummary)

    # Establish a list of indices that will used to identify our training and testing data with a 60-40 split
    dataset_index = createDataPartition(y = dataset[[outcome]], p = 0.6, list = FALSE)
    
    # Use indices to make our training and testing datasets and view the number of Ds and NDs
    data_train = dataset[dataset_index,]
    
    # using SMOTE to balance the class distribution
    balanced_data_train = smotenc(data_train, outcome)
    data_test = dataset[-dataset_index,]

    # now pruning parameters (based on the training dataset to prevent overfitting)
    svm_tune = train(form = MCL~., data = balanced_data_train, method = model, trControl = control,
         metric = "ROC", # this gives me the same as balanced accuracy but it isn't able to find that ?
         maximize = TRUE, tuneGrid = tunegrid) 

    # predicting with tune parameters
    data_test[[pred_outcome]] = predict(svm_tune, newdata = data_test)
    
    # always specify the correct positive class when calculating the confusion matrix
    matrix = confusionMatrix(data = data_test[[pred_outcome]], reference = data_test[[outcome]],
                                 positive = "D")

    #calculating AUC
    AUC = auc(response = data_test[[outcome]], predictor = factor(data_test[[pred_outcome]], ordered = TRUE))
    
    #extracting accuracy, sens, spec, PPV, NPV to take mean later
    metrics = data.frame(t(c(matrix$byClass[11])), t(c(matrix$byClass[1:4])), AUC) %>%
        rename(PPV = Pos.Pred.Value, NPV = Neg.Pred.Value) %>%
        mutate(Model = "SVM", `Use_Case` = use_case, Kernel = model)  

    metrics = metrics[,c(7:9,1:6)]

    # getting variable importance
    svm_tune = svm_tune$finalModel

    coefs = svm_tune@coef[[1]]
    mat = svm_tune@xmatrix[[1]]
    
    variable_importance_df = data.frame(t(coefs %*% mat)) %>%
        rename(`Variable Importance` = t.coefs.....mat.) %>%
        arrange(-`Variable Importance`) %>%
        mutate(`Use Case` = use_case, Kernel = model) %>%
        rownames_to_column(var = "Variable")
    
    return(list(metrics, variable_importance_df))

}

In [14]:
#calling fn
svm_case_1_linear = svm_classification(manganese_case_1_df, "svmLinear", "MCL", "pred_MCL",
                                            tunegrid_linear_svm, 1)
svm_case_1_radial = svm_classification(manganese_case_1_df, "svmRadial", "MCL", "pred_MCL",
                                            NULL, 1)
svm_case_1_polynomial = svm_classification(manganese_case_1_df, "svmPoly", "MCL", "pred_MCL",
                                            NULL, 1)
svm_case_2_linear = svm_classification(manganese_case_2_df, "svmLinear", "MCL", "pred_MCL",
                                            tunegrid_linear_svm, 2)
svm_case_2_radial = svm_classification(manganese_case_2_df, "svmRadial", "MCL", "pred_MCL",
                                            NULL, 2)
svm_case_2_polynomial = svm_classification(manganese_case_2_df, "svmPoly", "MCL", "pred_MCL",
                                            NULL, 2)
svm_case_3_linear = svm_classification(manganese_case_3_df, "svmLinear", "MCL", "pred_MCL",
                                            tunegrid_linear_svm, 3)
svm_case_3_radial = svm_classification(manganese_case_3_df, "svmRadial", "MCL", "pred_MCL",
                                            NULL, 3)
svm_case_3_polynomial = svm_classification(manganese_case_3_df, "svmPoly", "MCL", "pred_MCL",
                                            NULL, 3)
svm_case_4_linear = svm_classification(manganese_case_4_df, "svmLinear", "MCL", "pred_MCL",
                                            tunegrid_linear_svm, 4)
svm_case_4_radial = svm_classification(manganese_case_4_df, "svmRadial", "MCL", "pred_MCL",
                                            NULL, 4)
svm_case_4_polynomial = svm_classification(manganese_case_4_df, "svmPoly", "MCL", "pred_MCL",
                                            NULL, 4)

Setting levels: control = ND, case = D

Warning message in value[[3L]](cond):
“Ordered predictor converted to numeric vector. Threshold values will not correspond to values in predictor.”
Setting direction: controls < cases

Setting levels: control = ND, case = D

Warning message in value[[3L]](cond):
“Ordered predictor converted to numeric vector. Threshold values will not correspond to values in predictor.”
Setting direction: controls < cases

Setting levels: control = ND, case = D

Warning message in value[[3L]](cond):
“Ordered predictor converted to numeric vector. Threshold values will not correspond to values in predictor.”
Setting direction: controls < cases

Setting levels: control = ND, case = D

Warning message in value[[3L]](cond):
“Ordered predictor converted to numeric vector. Threshold values will not correspond to values in predictor.”
Setting direction: controls < cases

Setting levels: control = ND, case = D

Warning message in value[[3L]](cond):
“Ordered predictor con

Rerunning SVM models with noise variables to determine which predictors are significant above the background noise. This will only be done for use case 1 and 2 with a linear kernel.

In [ ]:
train_vars_noise_manganese_case_1 = manganese_case_1_df
train_vars_noise_manganese_case_2 = manganese_case_2_df

noise_df = function(train_vars_noise){
    set.seed(8)
    # Add random noise predictors as an additional method to evaluate model performance
    # Adding a column that contains randomly shuffled values from one of the molecules; sampling with replacement
    train_vars_noise$noise1 = sample(train_vars_noise[[colnames(train_vars_noise[7])]], replace = TRUE) 
    train_vars_noise$noise2 = sample(train_vars_noise[[colnames(train_vars_noise[2])]], replace = TRUE)
    train_vars_noise$noise3 = sample(train_vars_noise[[colnames(train_vars_noise[5])]], replace = TRUE)
    train_vars_noise$noise4 = sample(train_vars_noise[[colnames(train_vars_noise[6])]], replace = TRUE)
    train_vars_noise$noise5 = sample(train_vars_noise[[colnames(train_vars_noise[9])]], replace = TRUE)
    
    return(train_vars_noise)
}

# calling fn
noise_training_dataset_manganese_case_1 = noise_df(train_vars_noise_manganese_case_1)
noise_training_dataset_manganese_case_2 = noise_df(train_vars_noise_manganese_case_2)

head(noise_training_dataset_manganese_case_1)

In [ ]:
# calling fn
noise_values_manganese_case_1 = svm_classification(noise_training_dataset_manganese_case_1, "svmLinear", "MCL", "pred_MCL",
                                            tunegrid_linear_svm, 1)
noise_values_manganese_case_2 = svm_classification(noise_training_dataset_manganese_case_2, "svmLinear", "MCL", "pred_MCL",
                                            tunegrid_linear_svm, 2)

In [ ]:
# viewing variable importance results
noise_confusion_matrix = data.frame(Model = "Linear SVM w/ Noise",
            rbind(noise_values_manganese_case_1[[1]], noise_values_manganese_case_2[[1]]))

noise_confusion_matrix

Determining which predictors ranked higher than the highest noise rank.

In [ ]:
noise_importance_values_case_1 = noise_values_manganese_case_1[[2]]
noise_importance_values_case_2 = noise_values_manganese_case_2[[2]] 

noise_variable_importance_ranks = function(noise_importance_values){
    # Determining what variables fall above the random noise variable importance ranks
    # :parameters: a dataframe containing the initial feature importance values, features, and use case
    # :output: a dataframe containing the feature, variable importance, significance, and use case
    
    # first we need to take the aboslute value since this is based solely off of magnitude
    noise_importance_df = noise_importance_values[,1:2] %>%
        mutate(`Variable Importance` = abs(`Variable Importance`)) %>%
        arrange(-`Variable Importance`) %>%
        mutate(Rank = 1:nrow(noise_importance_values)) %>%
        filter(grepl("noise", Predictor))
    
    # figuring out the position of the highest background noise importance value
    highest_noise_rank = noise_importance_df$Rank[1]

    # adding a column denoting if the feature was above random noise
    final_df = noise_importance_values %>%
        # the rank is based on the magnitudes so I need to get those again
        mutate(Magnitude = abs(`Variable Importance`)) %>%
        arrange(-Magnitude) %>%
        select(-Magnitude) %>%
        # removing noise variables
        filter(!grepl("noise", Predictor)) %>%
        arrange(-`Variable Importance`)
    
    for (i in 1:length(final_df$Predictor)){
        # if the position of the feature is above the highest noise then yes it passed the filter
        if(i < highest_noise_rank){
            final_df$Passed_Filter[i] = "Yes"
            
        # if not it didn't
        } else{
            final_df$Passed_Filter[i] = "No"
            }
        }
    
    return(final_df)
}

# calling fn
significant_predictors_df_case_1 = noise_variable_importance_ranks(noise_importance_values_case_1)
significant_predictors_df_case_2 = noise_variable_importance_ranks(noise_importance_values_case_2)

head(significant_predictors_df_case_1)

In [ ]:
# creating final confusion matrix df
final_df = rbind(rf_confusion_matrix, svm_case_1_linear[[1]], svm_case_1_radial[[1]], svm_case_1_polynomial[[1]], 
                 svm_case_2_linear[[1]], svm_case_2_radial[[1]], svm_case_2_polynomial[[1]], svm_case_3_linear[[1]], 
                 svm_case_3_radial[[1]], svm_case_3_polynomial[[1]], svm_case_4_linear[[1]], svm_case_4_radial[[1]], 
                 svm_case_4_polynomial[[1]]) 

final_df

In [ ]:
# creating a final SVM var importance df
svm_variable_importance_df = rbind(significant_predictors_df_case_1, svm_case_1_radial[[2]], svm_case_1_polynomial[[2]], 
                 significant_predictors_df_case_2, svm_case_2_radial[[2]], svm_case_2_polynomial[[2]], svm_case_3_linear[[2]], 
                 svm_case_3_radial[[2]], svm_case_3_polynomial[[2]], svm_case_4_linear[[2]], svm_case_4_radial[[2]], 
                 svm_case_4_polynomial[[2]])

head(svm_variable_importance_df)

In [ ]:
# exporting
write.xlsx(final_df, paste0(Output,"/", "Mn_Prediction_Confusion_Matrix_", cur_date, ".xlsx"), 
           rowNames = FALSE)
write.xlsx(significant_predictors_df, paste0(Output,"/", "Mn_RF_Variable_Importance_", cur_date, ".xlsx"), rowNames = FALSE)
write.xlsx(svm_variable_importance_df, paste0(Output,"/", "Mn_SVM_Variable_Importance_", cur_date, ".xlsx"), rowNames = FALSE)